In [3]:
#import libs

import pandas as pd
import os

In [ ]:
# В папке subsid (shared/homeworks/python_ds_miniprojects/5_subsid) находятся файлы (tm_sales_1, tm_sales_2, ...) с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

# Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

# Особенности данных:

# сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
# поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
# продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
# если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается
# Сохраните результат в датафрэйм с разделителем ;, содержащий корректные подключения.

In [33]:
prod_activations_logs = pd.read_csv('/home/jupyter-n-korotkov/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/prod_activations_logs.csv', sep=';')

In [17]:
tm_sales_all = pd.DataFrame()
for path, dic, file in os.walk('/home/jupyter-n-korotkov/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/'):
    for i in file:
        if i == 'prod_activations_logs.csv':
            continue
        else:
            temp_df = pd.read_csv(f'/home/jupyter-n-korotkov/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/{i}', sep=';')
            tm_sales_all = pd.concat([tm_sales_all, temp_df])

In [18]:
tm_sales_all.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
0,id5416547,3,1743,17-03-2020 10:17
1,NaN,4,7421,12-03-2020 11:25


In [19]:
tm_sales_all.dtypes

SUBS_ID      object
FILIAL_ID     int64
PROD_ID       int64
ACT_DTTM     object
dtype: object

In [20]:
# Lets clear and clarify the date time and reset indexes after that

tm_sales_all.ACT_DTTM = pd.to_datetime(tm_sales_all.ACT_DTTM, format='%d-%m-%Y %H:%M')
tm_sales_all = tm_sales_all.reset_index(drop=True)

In [21]:
tm_sales_all.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,2020-03-20 14:59:00
1,7646509,5,6431,2020-03-19 13:00:00
2,id7412683,4,3313,2020-03-22 17:25:00
3,id5416547,3,1743,2020-03-17 10:17:00
4,NaN,4,7421,2020-03-12 11:25:00


In [22]:
# Then drop NaN from DF
tm_sales_all = tm_sales_all.dropna()

In [25]:
# And ad 'id' to SUBS_ID
tm_sales_all['SUBS_ID'] = tm_sales_all['SUBS_ID'].str[-7:].apply(lambda name: 'id' + str(name))

In [26]:
tm_sales_all

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,2020-03-20 14:59:00
1,id7646509,5,6431,2020-03-19 13:00:00
2,id7412683,4,3313,2020-03-22 17:25:00
3,id5416547,3,1743,2020-03-17 10:17:00
5,id8362218,7,9879,2020-03-05 11:42:00
6,id2185490,2,3210,2020-03-16 16:28:00
7,id5764122,3,1499,2020-03-18 15:44:00
8,id7642700,6,3020,2020-03-15 14:21:00
9,id1374509,2,5677,2020-03-17 11:48:00


In [34]:
prod_activations_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
3,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
2,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
6,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
2,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [35]:
prod_activations_logs['START_DTTM'] = pd.to_datetime(prod_activations_logs['START_DTTM'], format='%d-%m-%Y %H:%M')

In [36]:
prod_activations_logs['END_DTTM'] = pd.to_datetime(prod_activations_logs['END_DTTM'], format='%d-%m-%Y %H:%M')

In [38]:
prod_activations_logs.dtypes

SUBS_ID               object
PROD_ID                int64
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

In [40]:
merged_data = tm_sales_all.merge(prod_activations_logs, on='SUBS_ID')

In [48]:
merged_data = merged_data.sort_values('SUBS_ID', ascending=True)

In [50]:
# Answer for a task.
merged_data.head(4).SUBS_ID.tolist()

['id2185490', 'id4651830', 'id5416547', 'id7642700']